# An introduction to JuMP

This tutorial is based on a tutorial presented at the [2019 Grid Science Winter School](https://github.com/lanl-ansi/tutorial-grid-science-2019)

This tutorial doesn't exist in isolation. Some other good resources for learning JuMP are
- [the Discourse forum](https://discourse.julialang.org/c/domain/opt)
- [JuMP documentation](http://www.juliaopt.org/JuMP.jl/v0.19.0/)
- [JuMP examples](https://github.com/JuliaOpt/JuMP.jl/tree/release-0.19/examples)
- [Textbook: Julia Programming for Operations Research](http://www.chkwon.net/julia/)

Before we start, run the following magic sauce to install the required Julia packages and check that we're good to go.

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Excellent! Everything is good to go!


## The basics

First, load the JuMP package into your current environment.

In [2]:
using JuMP, GLPK

Start building a JuMP model like so,

In [3]:
model = Model()
@variable(model, x)
@variable(model, y >= 0)
@variable(model, 1 <= z <= 2)
model

A JuMP Model
Feasibility problem with:
Variables: 3
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x, y, z

### What's going on here?
`@variable(model, x)` does four things:
1. it adds an *optimization* variable to the model
2. it creates a *JuMP* variable that acts as a reference to the optimization variable in the model
3. it creates a *Julia* variable `x` that points to the JuMP variable
4. it stores a reference to the JuMP variable in the model with the name `:x`

In [4]:
model = Model()
@variable(model, x >= 1.414)

x

In [5]:
# x is a JuMP variable
typeof(x)

VariableRef

In [6]:
# We can bind the JuMP variable to a different Julia variable and set `x` to something else
y = x
x = 1

@show typeof(y)
@show typeof(x)

y

typeof(y) = VariableRef
typeof(x) = Int64


x

In [7]:
JuMP.lower_bound(y)

1.414

In [8]:
model[:x]

x

In [9]:
model[:x] == y

true

### Other ways to create variables

We can also create arrays of JuMP variables.

In [10]:
model = Model()
@variable(model, x[i = 1:4] >= i)
model

A JuMP Model
Feasibility problem with:
Variables: 4
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

The indices of the arrays don't have to be integers. They can be anything, like a string or a symbol.

In [11]:
model = Model()
@variable(model, x[i = 1:2, j = [:A, :B]] >= i)
model

A JuMP Model
Feasibility problem with:
Variables: 4
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

What if I want to add two variables with the same name?

In [12]:
model = Model()
@variable(model, x >= 1)
@variable(model, x >= 2)

ErrorException: An object of name x is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

### Quiz Question

What is the value of the following?

In [13]:
JuMP.lower_bound(x)

1.0

### Constraints

Now that we've seen how to create variables, let's look at constraints. Much of the syntax should be familiar.

In [14]:
model = Model()
@variable(model, x >= 0)
@variable(model, y >= 0)

@constraint(model, c_less_than, 2x + y <= 1)
@constraint(model, c_greater_than, 2x + y >= 1)
@constraint(model, c_equal_to, 2x + y == 1)

model

A JuMP Model
Feasibility problem with:
Variables: 2
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: c_equal_to, c_greater_than, c_less_than, x, y

In [15]:
model[:c_equal_to]

c_equal_to : 2 x + y = 1.0

In [16]:
anonymous_constraint = @constraint(model, [i = 1:2], i * x <= y)

model

A JuMP Model
Feasibility problem with:
Variables: 2
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: c_equal_to, c_greater_than, c_less_than, x, y

### Objective Functions

Now let's look at the objective function.

In [17]:
model = Model()
@variable(model, x >= 0)

@objective(model, Min, 2x + 1)

model

A JuMP Model
Minimization problem with:
Variable: 1
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

In [18]:
model = Model()
@variable(model, x <= 2)

@objective(model, Max, 2x + 1)

model

A JuMP Model
Maximization problem with:
Variable: 1
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x

### Solving a Model

Once we've formulated a model, the next step is to solve it. This requires a solver.

JuMP supports lots of different solvers. The [JuMP documentation](http://www.juliaopt.org/JuMP.jl/v0.19.0/installation/#Getting-Solvers-1) contains a list of the supported solvers and the types of problems each solver supports.

We're going to use two solvers in particular.

The first solver is the [GNU Linear Programming Kit (GLPK)](https://www.gnu.org/software/glpk/). This solver supports linear programs with continous variables.

GLPK is available via the [GLPK.jl](https://github.com/JuliaOpt/GLPK.jl) package.

We are going to start with a simple example:

$$
\begin{align}
%
\max_{x \geq 0} & \quad 4x_1 + 3x_2\\
%
\mbox{s.t.: } & \nonumber \\
& 2x_1 + x_2 \leq 4 \\
& x_1 + 2x_2 \leq 4 \\
%
\end{align}
$$

In [19]:
production_model = Model(with_optimizer(GLPK.Optimizer))

#TODO
## variables

## constraints

## objective

production_model

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [20]:
optimize!(production_model)

glp_add_cols: ncs = 0; invalid number of columns
Error detected in file api/prob1.c at line 362


GLPKFatalError: GLPKFatalError("GLPK call failed. All GLPK objects you defined so far are now invalidated.")

In [21]:
x_val = JuMP.value.(x)
objval = JuMP.objective_value(production_model)

@show x_val
@show objval

NoOptimizer: NoOptimizer()

Another example
$$
\begin{align}
%
\min_{x, y \geq 0} & \quad \sum_{i = 1}^{10} x_i + y_i\\
%
\mbox{s.a.: } & \nonumber \\
& x_i + y_i \geq 1 \quad  \forall i = 1 \dots 10 \\
& y_i = 0.7 \quad  \forall i = 1 \dots 10 \\
& 0.3 \leq x_i \leq 0.5 \quad  \forall i = 1 \dots 10 \\
%
\end{align}
$$

In [22]:
model1 = Model(with_optimizer(GLPK.Optimizer))

#TODO
## variables

## constraints

## objective


model1

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [23]:
optimize!(model1)

glp_add_cols: ncs = 0; invalid number of columns
Error detected in file api/prob1.c at line 362


GLPKFatalError: GLPKFatalError("GLPK call failed. All GLPK objects you defined so far are now invalidated.")

In [24]:
x_val = JuMP.value.(x)
y_val = JuMP.value.(y)
objval = JuMP.objective_value(model1)
objval = JuMP.termination_status(model1)

@show x_val
@show y_val
@show objval

NoOptimizer: NoOptimizer()

What happen if we change the bounds of variable $x$ ?
$$
\begin{align}
%
\min_{x, y \geq 0} & \quad \sum_{i = 1}^{10} x_i + y_i\\
%
\mbox{s.a.: } & \nonumber \\
& x_i + y_i \geq 1 \quad  \forall i = 1 \dots 10 \\
& y_i = 0.7 \quad  \forall i = 1 \dots 10 \\
& 0.1 \leq x_i \leq 0.2 \quad  \forall i = 1 \dots 10 \\
%
\end{align}
$$

In [25]:
model2 = Model(with_optimizer(GLPK.Optimizer))

#TODO
## variables

## constraints

## objective


model2

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [26]:
optimize!(model2)

glp_add_cols: ncs = 0; invalid number of columns
Error detected in file api/prob1.c at line 362


GLPKFatalError: GLPKFatalError("GLPK call failed. All GLPK objects you defined so far are now invalidated.")

In [27]:
x_val = JuMP.value.(x)
y_val = JuMP.value.(y)
objval = JuMP.objective_value(model2)
objval = JuMP.termination_status(model2)

@show x_val
@show y_val
@show objval

NoOptimizer: NoOptimizer()